#### Bibliotecas e configuracoes

In [1]:
import os
import numpy as np
import pandas as pd
import seaborn as sns
import warnings
import concurrent.futures
from concurrent.futures import ThreadPoolExecutor

# Pacote com todas as funcoes devenvolvidas para esse projeto
from functions_proj_scraping import *

# Configs
pd.set_option('display.float_format', '{:.2f}'.format)
pd.set_option('display.precision', 2)
warnings.filterwarnings('ignore')

#### Pegar todos os anuncios de todas as paginas do site (macro) com paralelismo

In [2]:
url = 'https://www.lagoimobiliaria.com.br/pesquisa-de-imoveis/?locacao_venda=V&id_cidade%5B%5D=90&id_tipo_imovel%5B%5D=5&id_tipo_imovel%5B%5D=20&id_tipo_imovel%5B%5D=19&finalidade=residencial&dormitorio=0&garagem=0&vmi=&vma='
#url = 'https://www.lagoimobiliaria.com.br/pesquisa-de-imoveis/?locacao_venda=V&id_cidade%5B%5D=90&id_tipo_imovel%5B%5D=4&finalidade=0&dormitorio=0&garagem=0&vmi=&vma='
paginas = total_paginas_site(url)

base_geral = []

# Funcao para executar o scrapping
def processar_pagina(pag):
    soup = gerando_soup_principal(url, pag)
    pagina = soup.find('div', {'class':'properties_item_two row'}).findAll('div', class_='col-lg-3 col-md-4 col-sm-6 col-xs-12')
    
    resultados_pagina = []
    
    for item in range(0, len(pagina)):
        resultado = dados_pag_principal(pagina[item], item)
        resultados_pagina.append(resultado)
    
    return resultados_pagina

# Paralelismo
with concurrent.futures.ThreadPoolExecutor() as executor:
    resultados_paginas = list(executor.map(processar_pagina, range(paginas)))

# "Aplanhando" a lista de listas para obter uma única lista de resultados
base_geral = [resultado for resultados_pagina in resultados_paginas for resultado in resultados_pagina]

#### Carregando dados macro para dataframe e tratando dados

In [3]:
# Criando dataframe e Trocando Nan -> 0 e removendo duplicidades
df = pd.DataFrame(base_geral)
df.fillna(0, inplace=True)
df.drop_duplicates(subset=['link'], keep='first', inplace=True, ignore_index=True)

# Criando coluna com nro de indice para merge posterior
df['indice'] = df.index

# Corrigindo encoding no link para evitar erros
df['link'] = df.apply(lambda row: safeStr(row['link']), axis=1)

# Reindexando
df.reset_index(inplace=True, drop=True)

#### Pegando dados complementares de todos os anuncios com paralelismo

In [4]:
# Funcao para processar o scrapping de cada anuncio especificamente
def processar_anuncio(link, index):
    try:
        pagina = gerando_soup_anuncio(link)
        resultado = dados_pag_anuncio(pagina, index)
        if resultado != 'erro':
            return resultado
    except:
        pass
    return None

# Criando df com links para loop no paralelismo
links = pd.DataFrame(df['link'])

# Criando lista para gravar resultados
base_geral = []

# Usando paralelismo com limite de operacoes simultaneas
max_workers = os.cpu_count()
with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
    futures = {executor.submit(processar_anuncio, row['link'], index): index for index, row in links.iterrows()}

    # Coletando  e gravando os resultados na lista
    for future in concurrent.futures.as_completed(futures):
        resultado = future.result()
        if resultado:
            base_geral.append(resultado)

#### Carregando dados anuncios para dataframe, tratando dados e combinando bases

In [5]:
# Criando dataframe e trocando Nan -> 0
df2 = pd.DataFrame(base_geral) 
df2.fillna(0, inplace=True)

# Mesclando df + df2
base_final = df.merge(df2, how='inner', on='indice')
base_final.fillna(0, inplace=True)
del df, df2

#### Tratando base final

In [6]:
# Criando coluna de preco
# Existem 2 campos de precos no site e ainda a possibilidade de haver apenas preco de aluguel
# Regras aqui: Se valor >= valor_venda entao mantem valor, senao valor_venda
base_final['preco'] = np.where(base_final['valor']>=base_final['valor_venda'],
                               base_final['valor'],
                               base_final['valor_venda'])

# Regras aqui: Se preco == valor_aluguel entao zera valor (queremos apenas anuncios de venda), senao mantem preco. 
base_final['preco'] = np.where(base_final['preco']==base_final['valor_aluguel'],
                               0,
                               base_final['preco'])

# Arredondando centavos
base_final['preco'] = base_final['preco'].round(0)

# Criando faixa de valores
faixas = ['até 250k','250k a 500k','501k a 750k','751k a 1M','acima 1M']
base_final['faixas_precos'] = pd.cut(base_final['preco'], bins=[0,250000,500000,750000,1000000,8000000], labels=faixas)

# Criando colunas de metragens
# Para CASAS existe area total e terreno, para APTOS existe area util e area construida
base_final['area_total_ou_terreno'] = np.where(base_final['area_total']==0,
                                               base_final['area_terreno'],
                                               base_final['area_total'])

base_final['area_util_ou_construida'] = np.where(base_final['area_util']==0,
                                                 base_final['area_construida'],
                                                 base_final['area_util'])

base_final['check_area'] = base_final['area_total_ou_terreno'] + base_final['area_util_ou_construida']

# Excluindo valores zerados
base_final = base_final[base_final['preco'] > 0]
base_final = base_final[base_final['check_area'] > 0]
base_final = base_final[base_final['classificacao'] != '0']
base_final.drop(columns=['indice','valor_venda','valor','area_util','area_total','check_area',
                         'area_terreno','area_construida'], inplace=True)

# Reindexando
base_final.reset_index(inplace=True, drop=True)

# Convertendo ints
colunas = ['quartos','suites','banheiros','vagas']
base_final[colunas] = base_final[colunas].astype('int32')

# Convertendo objetos para categorias
colunas = ['bairro','cidade','classificacao','pagina']
base_final[colunas] = base_final[colunas].astype('category')

# Ajustando bytes de tipagens numericas
base_final = tipagem(base_final)

# Remocao de outliers
base_final = remove_outliers(base_final)

# Criando coluna de preco x metragem
base_final['preco_m2'] = [round((preco / area_total),0) if area_total > area_util else round((preco / area_util),0) 
                          for preco, area_total, area_util in zip(base_final['preco'], 
                                                                  base_final['area_total_ou_terreno'], 
                                                                  base_final['area_util_ou_construida'])]

# Adicionando coluna para simplificar contagens
base_final['registros'] = 1

# Reorganizando
base_final = base_final[['bairro','cidade','descricao','telefones','link','link_foto_capa','classificacao','pagina',
                         'faixas_precos', 'preco','preco_m2','quartos','suites','banheiros','vagas','itens_do_imovel', 
                         'itens_do_condominio','area_total_ou_terreno','area_util_ou_construida','valor_condominio', 
                         'valor_iptu','valor_aluguel','registros']]

# Criando ID
base_final['id'] = base_final.index

#### Exportando arquivos csv

In [7]:
base_itens_imovel = separa_itens(base_final[['id','itens_do_imovel']],'itens_do_imovel')
base_itens_cond = separa_itens(base_final[['id','itens_do_condominio']],'itens_do_condominio')

base_final.to_csv('base_final_anuncios.csv',sep=';', encoding='utf-8', index=False, header=True)
base_itens_imovel.to_csv('base_itens_imovel.csv',sep=';', encoding='utf-8', index=False, header=True)
base_itens_cond.to_csv('base_itens_cond.csv',sep=';', encoding='utf-8', index=False, header=True)

#### Plotando estatisticas descritivas das variaveis

In [8]:
#base_final = pd.read_csv('base_final_anuncios.csv', sep=';', encoding='utf-8')
#base_final['pagina'] = base_final['pagina'].astype('category')

In [9]:
def calc_frequency_table(base, col_categ, col_cont):
    total = base[col_cont].count()
    frequency = (base.groupby(by=col_categ, as_index=False)[col_cont].count().rename(columns={col_cont:'freq_abs'}))
    frequency['freq_acum'] = frequency['freq_abs'].cumsum()
    frequency['freq_relat'] = (frequency['freq_abs']/total)*100
    frequency['freq_relat_acum'] = frequency['freq_relat'].cumsum()
    frequency['total_records'] = total
    
    return frequency

In [10]:
frequency = calc_frequency_table(base_final,['classificacao','faixas_precos'],'registros')
frequency

,classificacao,faixas_precos,freq_abs,freq_acum,freq_relat,freq_relat_acum,total_records
0,Apartamento - Padrão,até 250k,3012,3012,32.24,32.24,9343
1,Apartamento - Padrão,250k a 500k,2222,5234,23.78,56.02,9343
2,Apartamento - Padrão,501k a 750k,520,5754,5.57,61.59,9343
3,Apartamento - Padrão,751k a 1M,175,5929,1.87,63.46,9343
4,Apartamento - Padrão,acima 1M,113,6042,1.21,64.67,9343
5,Casas - Condomínio,até 250k,19,6061,0.20,64.87,9343
6,Casas - Condomínio,250k a 500k,58,6119,0.62,65.49,9343
7,Casas - Condomínio,501k a 750k,67,6186,0.72,66.21,9343
8,Casas - Condomínio,751k a 1M,57,6243,0.61,66.82,9343
9,Casas - Condomínio,acima 1M,148,6391,1.58,68.40,9343


In [11]:
import os
print(os.cpu_count())

8
